In [1]:
%pip install scikit-posthocs
%pip install scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
from google.colab import drive
import numpy as np
from scipy import stats
import scikit_posthocs as sp
drive.mount('/content/drive')
%cd '/content/drive/My Drive/Programming/Data Science/PES-IMD/Aprendizado de Máquina/Trabalho/Trab_AM/gato-cachorro_classificacao_de_racas/'

Mounted at /content/drive
/content/drive/My Drive/Programming/Data Science/PES-IMD/Aprendizado de Máquina/Trabalho/Trab_AM/gato-cachorro_classificacao_de_racas


In [3]:
path_to_accuracies = './accuracies/'

# kNN

In [ ]:
df_knn = pd.read_csv(path_to_accuracies+'knn_accuracies.csv')
df_knn.set_index(['dataset','train/test_type'],inplace=True)
df_knn

1         2         3         4  \
dataset       train/test_type                                           
normal_d=1764 Holdout: 70/30   0.300000  0.300000  0.310000  0.283333   
              Holdout: 80/20   0.265000  0.285000  0.290000  0.260000   
              Holdout: 90/10   0.300000  0.310000  0.320000  0.300000   
              10-fold          0.339000  0.330000  0.342000  0.318000   
normal_d=900  Holdout: 70/30   0.396667  0.380000  0.380000  0.420000   
              Holdout: 80/20   0.395000  0.360000  0.410000  0.420000   
              Holdout: 90/10   0.420000  0.350000  0.390000  0.400000   
              10-fold          0.395000  0.377000  0.355000  0.355000   
normal_d=459  Holdout: 70/30   0.286667  0.286667  0.313333  0.300000   
              Holdout: 80/20   0.270000  0.260000  0.285000  0.270000   
              Holdout: 90/10   0.310000  0.290000  0.320000  0.310000   
              10-fold          0.338000  0.343000  0.334000  0.321000   
normal_d=540  Holdout: 70/30   0.313333  0.303333  0.290000  0.313333   
              Holdout: 80/20   0.310000  0.300000  0.255000  0.250000   
              Holdout: 90/10   0.290000  0.310000  0.250000  0.230000   
              10-fold          0.342000  0.323000  0.308000  0.308000   

                                      5  
dataset       train/test_type            
normal_d=1764 Holdout: 70/30   0.290000  
              Holdout: 80/20   0.245000  
              Holdout: 90/10   0.270000  
              10-fold          0.318000  
normal_d=900  Holdout: 70/30   0.386667  
              Holdout: 80/20   0.390000  
              Holdout: 90/10   0.370000  
              10-fold          0.347000  
normal_d=459  Holdout: 70/30   0.296667  
              Holdout: 80/20   0.245000  
              Holdout: 90/10   0.300000  
              10-fold          0.324000  
normal_d=540  Holdout: 70/30   0.306667  
              Holdout: 80/20   0.255000  
              Holdout: 90/10   0.270000  
              10-fold          0.331000

## Some statistics

In [ ]:
df_knn.describe()

,1,2,3,4,5
count,16.000000,16.000000,16.000000,16.000000,16.000000
mean,0.329417,0.319250,0.322021,0.316167,0.309063
std,0.048620,0.034729,0.045371,0.057113,0.047115
min,0.265000,0.260000,0.250000,0.230000,0.245000
25%,0.297500,0.297500,0.290000,0.280000,0.270000
50%,0.311667,0.310000,0.316667,0.309000,0.303333
75%,0.355250,0.344750,0.345250,0.329500,0.335000
max,0.420000,0.380000,0.410000,0.420000,0.390000


## Statistical test

In [ ]:
stats.friedmanchisquare(*[df_knn[col] for col in df_knn.columns])

FriedmanchisquareResult(statistic=12.295819935691306, pvalue=0.015281841796910242)

There is a chance of 1.53% of the null hypothesis (the populations are the same) being true. If our threshold to reject the null hipothesis is < 5%, we can conclude that there is enough evidence to reject the null hypothesis. Thus, we accept that the population of the four tested samples are different.

### Executing pos-hoc analysis

In [ ]:
nemenyi_p = sp.posthoc_nemenyi_friedman(np.array([df_knn[col] for col in df_knn.columns]).T)
nemenyi_p.index = [f'k={str(i)}' for i in range(1,6)]
nemenyi_p.columns = [f'k={str(i)}' for i in range(1,6)]
nemenyi_p

,k=1,k=2,k=3,k=4,k=5
k=1,1.000000,0.415233,0.900000,0.287661,0.017975
k=2,0.415233,1.000000,0.707384,0.900000,0.643932
k=3,0.900000,0.707384,1.000000,0.580482,0.075861
k=4,0.287661,0.900000,0.580482,1.000000,0.770836
k=5,0.017975,0.643932,0.075861,0.770836,1.000000


There is statistical difference between k=1,k=5 and k=3,k=5. But k = 1 abd k = 1 have greater mean accuracy than k = 5. Thus the choice of k will be made between k = 1 and k = 3. Using a unitary number of neighbors (k = 1) is not recommended (find reference), so the best choice is k = 3.

# KNN with n_neighboors = [1,20]

In [ ]:
df_knn_new = pd.read_csv(path_to_accuracies+'knn_accuracies_new.csv')
df_knn_new = df_knn_new.groupby('dataset').mean().T
df_knn_new

dataset,normal_d=1764,normal_d=459,normal_d=540,normal_d=900
1,0.301500,0.302167,0.313333,0.403417
2,0.308500,0.294917,0.311333,0.370750
3,0.313500,0.312583,0.277000,0.391500
4,0.291333,0.301000,0.273833,0.401500
5,0.281750,0.290667,0.287917,0.378667
6,0.293000,0.311583,0.286833,0.368750
7,0.295667,0.304750,0.291750,0.364083
8,0.279750,0.293833,0.297167,0.372250
9,0.292250,0.290000,0.293083,0.351417
10,0.288167,0.279167,0.288500,0.336417


In [ ]:
df_knn_new.describe()

dataset,normal_d=1764,normal_d=459,normal_d=540,normal_d=900
count,20.000000,20.000000,20.000000,20.000000
mean,0.283175,0.282867,0.284771,0.346242
std,0.014125,0.018137,0.013449,0.032003
min,0.263250,0.258083,0.266917,0.311917
25%,0.272479,0.265417,0.273521,0.319979
50%,0.279625,0.284042,0.284458,0.330083
75%,0.292437,0.296438,0.292083,0.371125
max,0.313500,0.312583,0.313333,0.403417


In [ ]:
stats.friedmanchisquare(*[df_knn_new[col] for col in df_knn_new.columns])

FriedmanchisquareResult(statistic=37.860000000000014, pvalue=3.0261015695770416e-08)

In [ ]:
nemenyi_p = sp.posthoc_nemenyi_friedman(np.array([df_knn_new[col] for col in df_knn_new.columns]).T)
nemenyi_p.index = df_knn_new.columns
nemenyi_p.columns = df_knn_new.columns
nemenyi_p

dataset,normal_d=1764,normal_d=459,normal_d=540,normal_d=900
dataset,,,,
normal_d=1764,1.000000,0.900000,0.803933,0.001
normal_d=459,0.900000,1.000000,0.528951,0.001
normal_d=540,0.803933,0.528951,1.000000,0.001
normal_d=900,0.001000,0.001000,0.001000,1.000


# DecisionTree

In [ ]:
df_dt = pd.read_csv(path_to_accuracies+'dt_accuracies.csv')
df_dt.set_index(['dataset','train/test_type'],inplace=True)
df_dt

3         4         5         6  \
dataset       train/test_type                                           
normal_d=1764 Holdout: 70/30   0.310000  0.286667  0.273333  0.296667   
              Holdout: 80/20   0.275000  0.325000  0.300000  0.310000   
              Holdout: 90/10   0.360000  0.340000  0.310000  0.290000   
              10-fold          0.313000  0.330000  0.337000  0.322000   
normal_d=900  Holdout: 70/30   0.310000  0.266667  0.283333  0.306667   
              Holdout: 80/20   0.360000  0.365000  0.355000  0.340000   
              Holdout: 90/10   0.280000  0.320000  0.350000  0.350000   
              10-fold          0.294000  0.341000  0.343000  0.333000   
normal_d=459  Holdout: 70/30   0.316667  0.296667  0.326667  0.300000   
              Holdout: 80/20   0.310000  0.310000  0.260000  0.295000   
              Holdout: 90/10   0.330000  0.300000  0.270000  0.260000   
              10-fold          0.350000  0.329000  0.314000  0.316000   
normal_d=540  Holdout: 70/30   0.293333  0.296667  0.290000  0.280000   
              Holdout: 80/20   0.270000  0.285000  0.290000  0.295000   
              Holdout: 90/10   0.330000  0.350000  0.300000  0.300000   
              10-fold          0.289000  0.310000  0.320000  0.331000   

                                      7  
dataset       train/test_type            
normal_d=1764 Holdout: 70/30   0.263333  
              Holdout: 80/20   0.290000  
              Holdout: 90/10   0.290000  
              10-fold          0.331000  
normal_d=900  Holdout: 70/30   0.326667  
              Holdout: 80/20   0.350000  
              Holdout: 90/10   0.330000  
              10-fold          0.325000  
normal_d=459  Holdout: 70/30   0.306667  
              Holdout: 80/20   0.285000  
              Holdout: 90/10   0.300000  
              10-fold          0.306000  
normal_d=540  Holdout: 70/30   0.280000  
              Holdout: 80/20   0.315000  
              Holdout: 90/10   0.320000  
              10-fold          0.324000

## Some statistics

In [ ]:
df_dt.describe()

,3,4,5,6,7
count,16.000000,16.000000,16.000000,16.000000,16.000000
mean,0.311937,0.315729,0.307646,0.307833,0.308917
std,0.028394,0.026520,0.029330,0.023401,0.022914
min,0.270000,0.266667,0.260000,0.260000,0.263333
25%,0.292250,0.296667,0.288333,0.295000,0.290000
50%,0.310000,0.315000,0.305000,0.303333,0.310833
75%,0.330000,0.332500,0.329250,0.324250,0.325417
max,0.360000,0.365000,0.355000,0.350000,0.350000


## Statistical test

In [ ]:
stats.friedmanchisquare(*[df_dt[col] for col in df_dt.columns])

FriedmanchisquareResult(statistic=2.0000000000000346, pvalue=0.7357588823428783)

There is a chance of 73.58% of the null hypothesis (the populations are the same) being true. If our threshold to reject the null hipothesis is < 5%, we can conclude that there is not enough evidence to reject the null hypothesis. Thus, we accept that the population of the four tested samples are the same. This doesn't proven that the two populations are the same. But for a lack of evidence on the contrary we accept that they are the same. So the best choice is dictated by the greatest mean value, that, in this case is for max_depth = 4.

# Decision Tree with max_depth = [3,23]

In [ ]:
df_dt_new = pd.read_csv(path_to_accuracies+'dt_accuracies_new.csv')
df_dt_new = df_dt_new.groupby('dataset').mean().T
df_dt_new

dataset,normal_d=1764,normal_d=459,normal_d=540,normal_d=900
3,0.309750,0.334167,0.294833,0.317250
4,0.313167,0.315917,0.314750,0.326500
5,0.291250,0.300917,0.305750,0.342167
6,0.297000,0.299917,0.288500,0.332583
7,0.299250,0.302917,0.299333,0.324083
8,0.298250,0.297417,0.279917,0.324667
9,0.307583,0.297583,0.293750,0.330333
10,0.313167,0.286500,0.305750,0.333167
11,0.280750,0.290500,0.307750,0.324250
12,0.295000,0.285833,0.315250,0.320417


In [ ]:
df_dt_new.describe()

dataset,normal_d=1764,normal_d=459,normal_d=540,normal_d=900
count,21.000000,21.000000,21.000000,21.000000
mean,0.297571,0.295032,0.301016,0.320321
std,0.009347,0.012951,0.010461,0.010602
min,0.280750,0.277583,0.279917,0.300417
25%,0.290833,0.286500,0.294083,0.314833
50%,0.297000,0.294250,0.304917,0.320417
75%,0.304167,0.299917,0.307750,0.326500
max,0.313167,0.334167,0.315250,0.342167


In [ ]:
stats.friedmanchisquare(*[df_dt_new[col] for col in df_dt_new.columns])

FriedmanchisquareResult(statistic=30.714285714285722, pvalue=9.763242243869743e-07)

In [ ]:
nemenyi_p = sp.posthoc_nemenyi_friedman(np.array([df_dt_new[col] for col in df_dt_new.columns]).T)
nemenyi_p.index = df_dt_new.columns
nemenyi_p.columns = df_dt_new.columns
nemenyi_p

dataset,normal_d=1764,normal_d=459,normal_d=540,normal_d=900
dataset,,,,
normal_d=1764,1.000000,0.900000,0.614262,0.001000
normal_d=459,0.900000,1.000000,0.614262,0.001000
normal_d=540,0.614262,0.614262,1.000000,0.001908
normal_d=900,0.001000,0.001000,0.001908,1.000000


# Naive Bayes

In [ ]:
df_nb = pd.read_csv(path_to_accuracies+'nb_accuracies.csv')
df_nb.rename(columns={'1e-09':'default'}, inplace = True)
df_nb.set_index(['dataset','train/test_type'],inplace=True)
df_nb

default
dataset       train/test_type          
normal_d=1764 Holdout: 70/30   0.463333
              Holdout: 80/20   0.495000
              Holdout: 90/10   0.480000
              10-fold          0.512000
normal_d=900  Holdout: 70/30   0.536667
              Holdout: 80/20   0.515000
              Holdout: 90/10   0.480000
              10-fold          0.529000
normal_d=459  Holdout: 70/30   0.373333
              Holdout: 80/20   0.415000
              Holdout: 90/10   0.490000
              10-fold          0.424000
normal_d=540  Holdout: 70/30   0.450000
              Holdout: 80/20   0.505000
              Holdout: 90/10   0.470000
              10-fold          0.505000

## Some statistics

In [ ]:
df_nb.describe()

,default
count,16.000000
mean,0.477708
std,0.044248
min,0.373333
25%,0.460000
50%,0.485000
75%,0.506750
max,0.536667


In [ ]:
df_nb_new =  df_nb.reset_index().groupby('dataset').aggregate(['mean','std'])
df_nb_new

default          
                   mean       std
dataset                          
normal_d=1764  0.487583  0.020791
normal_d=459   0.425583  0.048284
normal_d=540   0.482500  0.027234
normal_d=900   0.515167  0.025102

# MLP

## Activation

In [4]:
df_mlp_activation = pd.read_csv(path_to_accuracies+'MLP_actv_accuracies.csv')
df_mlp_activation

,Unnamed: 0,Unnamed: 1,MLP,MLP.1,MLP.2,MLP.3
0,NaN,NaN,activation=identity,activation=logistic,activation=tanh,activation=relu
1,dataset,train/test_type,NaN,NaN,NaN,NaN
2,normal_d=1764,70/30-HOLDOUT,0.49333333333333335,0.5133333333333333,0.5,0.5333333333333333
3,normal_d=1764,80/20-HOLDOUT,0.485,0.495,0.495,0.54
4,normal_d=1764,90/10-HOLDOUT,0.51,0.54,0.56,0.59
5,normal_d=1764,10-FOLD,0.5050000000000001,0.539,0.542,0.5630000000000001
6,normal_d=900,70/30-HOLDOUT,0.5,0.5233333333333333,0.5066666666666667,0.53
7,normal_d=900,80/20-HOLDOUT,0.505,0.55,0.545,0.605
8,normal_d=900,90/10-HOLDOUT,0.5,0.54,0.64,0.64
9,normal_d=900,10-FOLD,0.495,0.526,0.5339999999999999,0.5690000000000001


### Some statistics

In [ ]:
df_mlp_activation.describe()

,identity,logistic,tanh,relu
count,16.000000,16.000000,16.000000,16.000000
mean,0.479250,0.493125,0.503583,0.515917
std,0.023967,0.040500,0.034597,0.042105
min,0.416667,0.420000,0.426667,0.430000
25%,0.471500,0.469500,0.489167,0.479750
50%,0.482500,0.499000,0.502500,0.523500
75%,0.492000,0.520750,0.530000,0.553083
max,0.515000,0.555000,0.565000,0.569000


The best mean accuracy is obtained when the logistic function is used as activatition function. So the 'activation' mlp parameter will be set to 'relu' in the next analyzes.

## Hidden layer sizes

In [ ]:
df_mlp_hls = pd.read_csv(path_to_accuracies+'mlp_hidden_layer_sizes_accuracies.csv')
df_mlp_hls

,dataset,train/test_type,O,OA,A,AT,T
0,normal_d=1764,Holdout: 70/30,0.480000,0.493333,0.493333,0.493333,0.503333
1,normal_d=1764,Holdout: 80/20,0.405000,0.540000,0.530000,0.520000,0.560000
2,normal_d=1764,Holdout: 90/10,0.490000,0.530000,0.510000,0.520000,0.480000
3,normal_d=1764,10-fold,0.471000,0.563000,0.566000,0.558000,0.550000
4,normal_d=900,Holdout: 70/30,0.470000,0.586667,0.610000,0.580000,0.596667
5,normal_d=900,Holdout: 80/20,0.530000,0.620000,0.590000,0.585000,0.590000
6,normal_d=900,Holdout: 90/10,0.460000,0.630000,0.580000,0.590000,0.560000
7,normal_d=900,10-fold,0.466000,0.558000,0.573000,0.576000,0.581000
8,normal_d=459,Holdout: 70/30,0.423333,0.436667,0.490000,0.440000,0.450000
9,normal_d=459,Holdout: 80/20,0.410000,0.475000,0.440000,0.490000,0.470000


### Some statistics

In [ ]:
df_mlp_hls.describe()

,O,OA,A,AT,T
count,16.000000,16.000000,16.000000,16.000000,16.000000
mean,0.459646,0.528396,0.525438,0.519604,0.525417
std,0.035086,0.054269,0.047572,0.050132,0.047972
min,0.405000,0.436667,0.440000,0.440000,0.450000
25%,0.428333,0.491667,0.492500,0.492500,0.495000
50%,0.463000,0.527500,0.517000,0.520000,0.529000
75%,0.482500,0.559250,0.567750,0.562500,0.560000
max,0.530000,0.630000,0.610000,0.590000,0.596667


The best mean accuracy is obtained when the number of neurons is OA. So, besides the parameter used in the previous analysis, the 'hidden_layer_sizes' mlp parameter will also be setted to the 'OA' number - which number depends on the data set used for train/test - in the next analyzes.

## Max Iterations

In [ ]:
df_mlp_max_it = pd.read_csv(path_to_accuracies+'mlp_max_iter_accuracies.csv')
df_mlp_max_it

,dataset,train/test_type,100,1000,5000
0,normal_d=1764,Holdout: 70/30,0.503333,0.486667,0.490000
1,normal_d=1764,Holdout: 80/20,0.530000,0.530000,0.545000
2,normal_d=1764,Holdout: 90/10,0.500000,0.540000,0.500000
3,normal_d=1764,10-fold,0.561000,0.573000,0.559000
4,normal_d=900,Holdout: 70/30,0.586667,0.600000,0.596667
5,normal_d=900,Holdout: 80/20,0.615000,0.635000,0.570000
6,normal_d=900,Holdout: 90/10,0.530000,0.590000,0.540000
7,normal_d=900,10-fold,0.586000,0.571000,0.567000
8,normal_d=459,Holdout: 70/30,0.436667,0.410000,0.420000
9,normal_d=459,Holdout: 80/20,0.515000,0.480000,0.440000


### Some statistics

In [ ]:
df_mlp_max_it.describe()

,100,1000,5000
count,16.000000,16.000000,16.000000
mean,0.520583,0.522250,0.512979
std,0.050125,0.061526,0.049695
min,0.430000,0.410000,0.420000
25%,0.502500,0.483750,0.480000
50%,0.517500,0.522500,0.510000
75%,0.537750,0.571500,0.548500
max,0.615000,0.635000,0.596667


The best mean accuracy is obtained when the maximum number of iteration were 100, but the standard deviation is significantly greater the other ones. Also, there were some convergence issues when using this number of iterations. Thus, besides the parameters setted in the previous analysis, the 'max_iter' mlp parameter will also be setted to 1000 in the next analyzes.

## Learning rate

In [ ]:
df_mlp_lr = pd.read_csv(path_to_accuracies+'mlp_learning_rate_init_accuracies.csv')
df_mlp_lr

,dataset,train/test_type,0.001,0.01,0.1
0,normal_d=1764,Holdout: 70/30,0.490000,0.483333,0.446667
1,normal_d=1764,Holdout: 80/20,0.540000,0.525000,0.500000
2,normal_d=1764,Holdout: 90/10,0.500000,0.560000,0.470000
3,normal_d=1764,10-fold,0.559000,0.538000,0.505000
4,normal_d=900,Holdout: 70/30,0.610000,0.570000,0.526667
5,normal_d=900,Holdout: 80/20,0.565000,0.595000,0.510000
6,normal_d=900,Holdout: 90/10,0.540000,0.560000,0.560000
7,normal_d=900,10-fold,0.570000,0.578000,0.527000
8,normal_d=459,Holdout: 70/30,0.446667,0.450000,0.446667
9,normal_d=459,Holdout: 80/20,0.520000,0.525000,0.465000


### Some statistics

In [ ]:
df_mlp_lr.describe()

,0.001,0.01,0.1
count,16.000000,16.000000,16.000000
mean,0.520021,0.517229,0.482417
std,0.044774,0.050418,0.037595
min,0.446667,0.410000,0.430000
25%,0.486667,0.490833,0.452167
50%,0.525000,0.520000,0.475500
75%,0.544750,0.560000,0.506250
max,0.610000,0.595000,0.560000


The best mean accuracy is obtained when the learning rate were 0.001.

## Final

In [ ]:
df_mlp = pd.DataFrame()
df_mlp['dataset'] = df_mlp_lr['dataset']
df_mlp['train/test_type'] = df_mlp_lr['train/test_type']
df_mlp['activation:relu'] = df_mlp_activation['relu']
df_mlp['hls:OA'] = df_mlp_hls['OA']
df_mlp['max_it:1000'] = df_mlp_max_it['1000']
df_mlp['learning_rate:0.001'] = df_mlp_lr['0.001']
df_mlp.set_index(['dataset','train/test_type'],inplace=True)
df_mlp

activation:relu    hls:OA  max_it:1000  \
dataset       train/test_type                                           
normal_d=1764 Holdout: 70/30          0.473333  0.493333     0.486667   
              Holdout: 80/20          0.515000  0.540000     0.530000   
              Holdout: 90/10          0.490000  0.530000     0.540000   
              10-fold                 0.553000  0.563000     0.573000   
normal_d=900  Holdout: 70/30          0.553333  0.586667     0.600000   
              Holdout: 80/20          0.560000  0.620000     0.635000   
              Holdout: 90/10          0.540000  0.630000     0.590000   
              10-fold                 0.569000  0.558000     0.571000   
normal_d=459  Holdout: 70/30          0.430000  0.436667     0.410000   
              Holdout: 80/20          0.555000  0.475000     0.480000   
              Holdout: 90/10          0.520000  0.460000     0.450000   
              10-fold                 0.479000  0.505000     0.485000   
normal_d=540  Holdout: 70/30          0.480000  0.486667     0.503333   
              Holdout: 80/20          0.550000  0.525000     0.515000   
              Holdout: 90/10          0.460000  0.510000     0.450000   
              10-fold                 0.527000  0.535000     0.537000   

                               learning_rate:0.001  
dataset       train/test_type                       
normal_d=1764 Holdout: 70/30              0.490000  
              Holdout: 80/20              0.540000  
              Holdout: 90/10              0.500000  
              10-fold                     0.559000  
normal_d=900  Holdout: 70/30              0.610000  
              Holdout: 80/20              0.565000  
              Holdout: 90/10              0.540000  
              10-fold                     0.570000  
normal_d=459  Holdout: 70/30              0.446667  
              Holdout: 80/20              0.520000  
              Holdout: 90/10              0.460000  
              10-fold                     0.508000  
normal_d=540  Holdout: 70/30              0.476667  
              Holdout: 80/20              0.535000  
              Holdout: 90/10              0.470000  
              10-fold                     0.530000

### Some statistics

In [ ]:
df_mlp.describe()

,activation:relu,hls:OA,max_it:1000,learning_rate:0.001
count,16.000000,16.000000,16.000000,16.000000
mean,0.515917,0.528396,0.522250,0.520021
std,0.042105,0.054269,0.061526,0.044774
min,0.430000,0.436667,0.410000,0.446667
25%,0.479750,0.491667,0.483750,0.486667
50%,0.523500,0.527500,0.522500,0.525000
75%,0.553083,0.559250,0.571500,0.544750
max,0.569000,0.630000,0.635000,0.610000


### Statistical tests

In [ ]:
stats.friedmanchisquare(*[df_mlp[col] for col in df_mlp.columns])

FriedmanchisquareResult(statistic=5.789808917197429, pvalue=0.12229651099371375)

There is a chance of 12.23% of the null hypothesis (the populations are the same) being true. If our threshold to reject the null hipothesis is < 5%, we can conclude that there is not enough evidence to reject the null hypothesis. Thus, we accept that the population of the four tested samples are the same.

In [ ]:
nemenyi_p = sp.posthoc_nemenyi_friedman(np.array([df_mlp[col] for col in df_mlp.columns]).T)
nemenyi_p.index = [col for col in df_mlp.columns]
nemenyi_p.columns = [col for col in df_mlp.columns]
nemenyi_p

,activation:relu,hls:OA,max_it:1000,learning_rate:0.001
activation:relu,1.000000,0.145912,0.317621,0.220908
hls:OA,0.145912,1.000000,0.900000,0.900000
max_it:1000,0.317621,0.900000,1.000000,0.900000
learning_rate:0.001,0.220908,0.900000,0.900000,1.000000


# MLP new

In [ ]:
df_mlp_new = pd.concat([df_mlp_activation,
                        df_mlp_hls[df_mlp_hls.columns.drop(['dataset',
                                                            'train/test_type'])],
                        df_mlp_max_it[['1000']],
                        df_mlp_lr[['0.01']]], axis=1)
df_mlp_new = df_mlp_new.groupby('dataset').mean().T
df_mlp_new

dataset,normal_d=1764,normal_d=459,normal_d=540,normal_d=900
identity,0.488667,0.463417,0.471250,0.493667
logistic,0.501583,0.454000,0.477000,0.539917
tanh,0.507667,0.480417,0.484250,0.542000
relu,0.507833,0.496000,0.504250,0.555583
O,0.461500,0.426833,0.468750,0.481500
OA,0.531583,0.469167,0.514167,0.598667
A,0.524833,0.476000,0.512667,0.588250
AT,0.522833,0.470750,0.502083,0.582750
T,0.523333,0.470750,0.525667,0.581917
1000,0.532417,0.456250,0.501333,0.599000


In [ ]:
df_mlp_new.describe()

dataset,normal_d=1764,normal_d=459,normal_d=540,normal_d=900
count,11.000000,11.000000,11.000000,11.000000
mean,0.511712,0.466758,0.496114,0.558091
std,0.021592,0.017490,0.018643,0.040409
min,0.461500,0.426833,0.468750,0.481500
25%,0.504625,0.459833,0.480625,0.540958
50%,0.522833,0.470750,0.501333,0.575750
75%,0.525708,0.473375,0.508458,0.585500
max,0.532417,0.496000,0.525667,0.599000


In [ ]:
stats.friedmanchisquare(*[df_mlp_new[col] for col in df_mlp_new.columns])

FriedmanchisquareResult(statistic=31.03636363636363, pvalue=8.35192865288927e-07)

In [ ]:
nemenyi_p = sp.posthoc_nemenyi_friedman(np.array([df_mlp_new[col] for col in df_mlp_new.columns]).T)
nemenyi_p.index = df_mlp_new.columns
nemenyi_p.columns = df_mlp_new.columns
nemenyi_p

dataset,normal_d=1764,normal_d=459,normal_d=540,normal_d=900
dataset,,,,
normal_d=1764,1.000000,0.005294,0.636277,0.138347
normal_d=459,0.005294,1.000000,0.138347,0.001000
normal_d=540,0.636277,0.138347,1.000000,0.005294
normal_d=900,0.138347,0.001000,0.005294,1.000000


# Statistical test between best samples of each model

## Friedman test

In [ ]:
stats.friedmanchisquare(
df_knn['3'],
df_dt['4'],
df_nb['default'],
df_mlp['learning_rate:0.001'])

FriedmanchisquareResult(statistic=41.9056603773585, pvalue=4.2013966776329e-09)

## Pos-hoc test

In [ ]:
nemenyi_p = sp.posthoc_nemenyi_friedman(np.array([df_knn['3'],
df_dt['4'],
df_nb['default'],
df_mlp['learning_rate:0.001']]).T)
nemenyi_p.index = ['knn','dt','nb','mlp']
nemenyi_p.columns = nemenyi_p.index
nemenyi_p

,knn,dt,nb,mlp
knn,1.000000,0.900000,0.004416,0.001000
dt,0.900000,1.000000,0.001625,0.001000
nb,0.004416,0.001625,1.000000,0.283091
mlp,0.001000,0.001000,0.283091,1.000000


# Bagging

## Knn

In [ ]:
df_bagging_knn = pd.read_csv(path_to_accuracies+'bagging_knn_accuracies.csv')
df_bagging_knn.set_index(['dataset','train/test_type'],inplace=True)
df_bagging_knn.columns = [f'knn/{col}'for col in df_bagging_knn.columns]
df_bagging_knn

knn/10    knn/20
dataset       train/test_type                    
normal_d=1764 Holdout: 70/30   0.310000  0.333333
              Holdout: 80/20   0.265000  0.270000
              Holdout: 90/10   0.330000  0.300000
              10-fold          0.325000  0.332000
normal_d=900  Holdout: 70/30   0.403333  0.396667
              Holdout: 80/20   0.400000  0.360000
              Holdout: 90/10   0.370000  0.370000
              10-fold          0.385000  0.401000
normal_d=459  Holdout: 70/30   0.300000  0.283333
              Holdout: 80/20   0.275000  0.275000
              Holdout: 90/10   0.310000  0.350000
              10-fold          0.349000  0.337000
normal_d=540  Holdout: 70/30   0.310000  0.293333
              Holdout: 80/20   0.285000  0.255000
              Holdout: 90/10   0.270000  0.320000
              10-fold          0.314000  0.299000

In [ ]:
df_bagging_knn.describe()

,knn/10,knn/20
count,16.000000,16.000000
mean,0.325083,0.323479
std,0.044820,0.044349
min,0.265000,0.255000
25%,0.296250,0.290833
50%,0.312000,0.326000
75%,0.354250,0.352500
max,0.403333,0.401000


### Statistical tests

In [ ]:
nemenyi_p = sp.posthoc_nemenyi_friedman(np.array([df_bagging_knn[col] for col in df_bagging_knn.columns]).T)
nemenyi_p.index = [i for i in df_bagging_knn.columns]
nemenyi_p.columns = [i for i in df_bagging_knn.columns]
nemenyi_p

,knn/10,knn/20
knn/10,1.000000,0.627173
knn/20,0.627173,1.000000


There is a chance of 62.72% of the null hypothesis (the populations are the same) being true. If our threshold to reject the null hipothesis is < 5%, we can conclude that there is not enough evidence to reject the null hypothesis. Thus, we accept that the population of the four tested samples are the same. This doesn't proven that the two populations are the same. But for a lack of evidence on the contrary we accept that they are the same. So the best choice is dictated by the greatest mean value, that, in this case is for n_estimators = 4.

## Decision Tree

In [ ]:
df_bagging_dt = pd.read_csv(path_to_accuracies+'bagging_dt_accuracies.csv')
df_bagging_dt.set_index(['dataset','train/test_type'],inplace=True)
df_bagging_dt.columns = [f'dt/{col}'for col in df_bagging_dt.columns]
df_bagging_dt

dt/10     dt/20
dataset       train/test_type                    
normal_d=1764 Holdout: 70/30   0.380000  0.433333
              Holdout: 80/20   0.455000  0.435000
              Holdout: 90/10   0.440000  0.430000
              10-fold          0.395000  0.437000
normal_d=900  Holdout: 70/30   0.376667  0.486667
              Holdout: 80/20   0.385000  0.455000
              Holdout: 90/10   0.420000  0.430000
              10-fold          0.434000  0.456000
normal_d=459  Holdout: 70/30   0.343333  0.376667
              Holdout: 80/20   0.385000  0.375000
              Holdout: 90/10   0.360000  0.360000
              10-fold          0.385000  0.409000
normal_d=540  Holdout: 70/30   0.423333  0.423333
              Holdout: 80/20   0.360000  0.415000
              Holdout: 90/10   0.410000  0.450000
              10-fold          0.376000  0.404000

### Some stats

In [ ]:
df_bagging_dt.describe()

,dt/10,dt/20
count,16.000000,16.000000
mean,0.395521,0.423500
std,0.031766,0.033143
min,0.343333,0.360000
25%,0.376500,0.407750
50%,0.385000,0.430000
75%,0.420833,0.440250
max,0.455000,0.486667


### Statistical tests

In [ ]:
nemenyi_p = sp.posthoc_nemenyi_friedman(np.array([df_bagging_dt[col] for col in df_bagging_dt.columns]).T)
nemenyi_p.index = [i for i in df_bagging_dt.columns]
nemenyi_p.columns = [i for i in df_bagging_dt.columns]
nemenyi_p

,dt/10,dt/20
dt/10,1.000000,0.045499
dt/20,0.045499,1.000000


There is a chance of 4.55% of the null hypothesis (the populations are the same) being true. If our threshold to reject the null hipothesis is < 5%, we can conclude that there is enough evidence to reject the null hypothesis. Thus, we reject that the population of the four tested samples are the same. So the best choice is dictated by the greatest mean value, that, in this case is for n_estimators = 20.

## Naive Bayes

In [ ]:
df_bagging_nb = pd.read_csv(path_to_accuracies+'bagging_nb_accuracies.csv')
df_bagging_nb.set_index(['dataset','train/test_type'],inplace=True)
df_bagging_nb.columns = [f'nb/{col}'for col in df_bagging_nb.columns]
df_bagging_nb

nb/10     nb/20
dataset       train/test_type                    
normal_d=1764 Holdout: 70/30   0.463333  0.453333
              Holdout: 80/20   0.490000  0.495000
              Holdout: 90/10   0.470000  0.500000
              10-fold          0.522000  0.508000
normal_d=900  Holdout: 70/30   0.543333  0.550000
              Holdout: 80/20   0.485000  0.510000
              Holdout: 90/10   0.470000  0.470000
              10-fold          0.527000  0.539000
normal_d=459  Holdout: 70/30   0.373333  0.386667
              Holdout: 80/20   0.415000  0.375000
              Holdout: 90/10   0.420000  0.460000
              10-fold          0.395000  0.398000
normal_d=540  Holdout: 70/30   0.453333  0.473333
              Holdout: 80/20   0.500000  0.490000
              Holdout: 90/10   0.470000  0.500000
              10-fold          0.494000  0.497000

### Some stats

In [ ]:
df_bagging_nb.describe()

,nb/10,nb/20
count,16.000000,16.000000
mean,0.468208,0.475333
std,0.047747,0.050890
min,0.373333,0.375000
25%,0.445000,0.458333
50%,0.470000,0.492500
75%,0.495500,0.502000
max,0.543333,0.550000


### Statistical tests

In [ ]:
nemenyi_p = sp.posthoc_nemenyi_friedman(np.array([df_bagging_nb[col] for col in df_bagging_nb.columns]).T)
nemenyi_p.index = [i for i in df_bagging_nb.columns]
nemenyi_p.columns = [i for i in df_bagging_nb.columns]
nemenyi_p

,nb/10,nb/20
nb/10,1.000000,0.080119
nb/20,0.080119,1.000000


There is a chance of 8.01% of the null hypothesis (the populations are the same) being true. If our threshold to reject the null hipothesis is < 5%, we can conclude that there is not enough evidence to reject the null hypothesis. Thus, we accept that the population of the four tested samples are the same. So the best choice is dictated by the greatest mean value, that, in this case is for n_estimators = 20.

## MLP

In [ ]:
df_bagging_mlp = pd.read_csv(path_to_accuracies+'bagging_mlp_accuracies.csv')
df_bagging_mlp.set_index(['dataset','train/test_type'],inplace=True)
df_bagging_mlp.columns = [f'mlp/{col}'for col in df_bagging_mlp.columns]
df_bagging_mlp

mlp/10    mlp/20
dataset       train/test_type                    
normal_d=1764 Holdout: 70/30   0.500000  0.506667
              Holdout: 80/20   0.525000  0.530000
              Holdout: 90/10   0.480000  0.540000
              10-fold          0.585000  0.576000
normal_d=900  Holdout: 70/30   0.616667  0.600000
              Holdout: 80/20   0.625000  0.640000
              Holdout: 90/10   0.620000  0.640000
              10-fold          0.575000  0.589000
normal_d=459  Holdout: 70/30   0.476667  0.460000
              Holdout: 80/20   0.515000  0.505000
              Holdout: 90/10   0.500000  0.450000
              10-fold          0.545000  0.553000
normal_d=540  Holdout: 70/30   0.486667  0.496667
              Holdout: 80/20   0.550000  0.550000
              Holdout: 90/10   0.520000  0.500000
              10-fold          0.540000  0.551000

### Some stats

In [ ]:
df_bagging_mlp.describe()

,mlp/10,mlp/20
count,16.000000,16.000000
mean,0.541250,0.542958
std,0.050172,0.056545
min,0.476667,0.450000
25%,0.500000,0.503750
50%,0.532500,0.545000
75%,0.577500,0.579250
max,0.625000,0.640000


### Statistical tests

In [ ]:
nemenyi_p = sp.posthoc_nemenyi_friedman(np.array([df_bagging_mlp[col] for col in df_bagging_mlp.columns]).T)
nemenyi_p.index = [i for i in df_bagging_mlp.columns]
nemenyi_p.columns = [i for i in df_bagging_mlp.columns]
nemenyi_p

,mlp/10,mlp/20
mlp/10,1.000000,0.453254
mlp/20,0.453254,1.000000


There is a chance of 4.53% of the null hypothesis (the populations are the same) being true. If our threshold to reject the null hipothesis is < 5%, we can conclude that there is enough evidence to reject the null hypothesis. Thus, we accept that the population of the four tested samples are the same. So the best choice is dictated by the greatest mean value, that, in this case is for n_estimators = 10, because the increase in n_estimator to 20 did not change the population and use n_estimators = 20 will result in more computational cost.

## Comparativo geral

In [ ]:
df_bagging_all = pd.concat([df_bagging_knn,df_bagging_dt,df_bagging_nb,df_bagging_mlp],axis=1)
df_bagging_all

knn/10    knn/20     dt/10     dt/20  \
dataset       train/test_type                                           
normal_d=1764 Holdout: 70/30   0.310000  0.333333  0.380000  0.433333   
              Holdout: 80/20   0.265000  0.270000  0.455000  0.435000   
              Holdout: 90/10   0.330000  0.300000  0.440000  0.430000   
              10-fold          0.325000  0.332000  0.395000  0.437000   
normal_d=900  Holdout: 70/30   0.403333  0.396667  0.376667  0.486667   
              Holdout: 80/20   0.400000  0.360000  0.385000  0.455000   
              Holdout: 90/10   0.370000  0.370000  0.420000  0.430000   
              10-fold          0.385000  0.401000  0.434000  0.456000   
normal_d=459  Holdout: 70/30   0.300000  0.283333  0.343333  0.376667   
              Holdout: 80/20   0.275000  0.275000  0.385000  0.375000   
              Holdout: 90/10   0.310000  0.350000  0.360000  0.360000   
              10-fold          0.349000  0.337000  0.385000  0.409000   
normal_d=540  Holdout: 70/30   0.310000  0.293333  0.423333  0.423333   
              Holdout: 80/20   0.285000  0.255000  0.360000  0.415000   
              Holdout: 90/10   0.270000  0.320000  0.410000  0.450000   
              10-fold          0.314000  0.299000  0.376000  0.404000   

                                  nb/10     nb/20    mlp/10    mlp/20  
dataset       train/test_type                                          
normal_d=1764 Holdout: 70/30   0.463333  0.453333  0.500000  0.506667  
              Holdout: 80/20   0.490000  0.495000  0.525000  0.530000  
              Holdout: 90/10   0.470000  0.500000  0.480000  0.540000  
              10-fold          0.522000  0.508000  0.585000  0.576000  
normal_d=900  Holdout: 70/30   0.543333  0.550000  0.616667  0.600000  
              Holdout: 80/20   0.485000  0.510000  0.625000  0.640000  
              Holdout: 90/10   0.470000  0.470000  0.620000  0.640000  
              10-fold          0.527000  0.539000  0.575000  0.589000  
normal_d=459  Holdout: 70/30   0.373333  0.386667  0.476667  0.460000  
              Holdout: 80/20   0.415000  0.375000  0.515000  0.505000  
              Holdout: 90/10   0.420000  0.460000  0.500000  0.450000  
              10-fold          0.395000  0.398000  0.545000  0.553000  
normal_d=540  Holdout: 70/30   0.453333  0.473333  0.486667  0.496667  
              Holdout: 80/20   0.500000  0.490000  0.550000  0.550000  
              Holdout: 90/10   0.470000  0.500000  0.520000  0.500000  
              10-fold          0.494000  0.497000  0.540000  0.551000

In [ ]:
df_bagging_all.describe()

,knn/10,knn/20,dt/10,dt/20,nb/10,nb/20,mlp/10,mlp/20
count,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000
mean,0.325083,0.323479,0.395521,0.423500,0.468208,0.475333,0.541250,0.542958
std,0.044820,0.044349,0.031766,0.033143,0.047747,0.050890,0.050172,0.056545
min,0.265000,0.255000,0.343333,0.360000,0.373333,0.375000,0.476667,0.450000
25%,0.296250,0.290833,0.376500,0.407750,0.445000,0.458333,0.500000,0.503750
50%,0.312000,0.326000,0.385000,0.430000,0.470000,0.492500,0.532500,0.545000
75%,0.354250,0.352500,0.420833,0.440250,0.495500,0.502000,0.577500,0.579250
max,0.403333,0.401000,0.455000,0.486667,0.543333,0.550000,0.625000,0.640000


In [ ]:
stats.friedmanchisquare(*[df_bagging_all[col] for col in df_bagging_all.columns])

FriedmanchisquareResult(statistic=102.17065868263472, pvalue=3.840941088057083e-19)

In [ ]:
algs = ['knn','dt','nb','mlp']
df_bagging_10 = df_bagging_all[[f'{alg}/10'for alg in algs]]
nemenyi_p = sp.posthoc_nemenyi_friedman(np.array([df_bagging_all[col] 
                                                  for col in df_bagging_10]).T)
nemenyi_p.index = [i.split('/')[0] for i in df_bagging_10.columns]
nemenyi_p.columns = [i.split('/')[0] for i in df_bagging_10.columns]
nemenyi_p

,knn,dt,nb,mlp
knn,1.000000,0.354855,0.001000,0.001000
dt,0.354855,1.000000,0.065622,0.001000
nb,0.001000,0.065622,1.000000,0.125527
mlp,0.001000,0.001000,0.125527,1.000000


In [ ]:
df_bagging_10 = df_bagging_all[[f'{alg}/20'for alg in algs]]
nemenyi_p = sp.posthoc_nemenyi_friedman(np.array([df_bagging_all[col] 
                                                  for col in df_bagging_10]).T)
nemenyi_p.index = [i.split('/')[0] for i in df_bagging_10.columns]
nemenyi_p.columns = [i.split('/')[0] for i in df_bagging_10.columns]
nemenyi_p

,knn,dt,nb,mlp
knn,1.000000,0.077891,0.001000,0.001000
dt,0.077891,1.000000,0.193413,0.001000
nb,0.001000,0.193413,1.000000,0.193413
mlp,0.001000,0.001000,0.193413,1.000000


# Boosting

## Decision Tree

In [ ]:
df_boosting_dt = pd.read_csv(path_to_accuracies+'boosting_dt_accuracies.csv')
df_boosting_dt.set_index(['dataset','train/test_type'],inplace=True)
df_boosting_dt.columns = [f'dt/{col}'for col in df_boosting_dt.columns]
df_boosting_dt

dt/10     dt/20
dataset       train/test_type                    
normal_d=1764 Holdout: 70/30   0.343333  0.320000
              Holdout: 80/20   0.365000  0.265000
              Holdout: 90/10   0.340000  0.230000
              10-fold          0.299000  0.305000
normal_d=900  Holdout: 70/30   0.353333  0.310000
              Holdout: 80/20   0.310000  0.295000
              Holdout: 90/10   0.310000  0.360000
              10-fold          0.333000  0.340000
normal_d=459  Holdout: 70/30   0.223333  0.226667
              Holdout: 80/20   0.235000  0.235000
              Holdout: 90/10   0.230000  0.180000
              10-fold          0.298000  0.275000
normal_d=540  Holdout: 70/30   0.256667  0.300000
              Holdout: 80/20   0.320000  0.295000
              Holdout: 90/10   0.300000  0.380000
              10-fold          0.321000  0.329000

### Some stats

In [ ]:
df_boosting_dt.describe()

,dt/10,dt/20
count,16.000000,16.000000
mean,0.302354,0.290354
std,0.044328,0.053091
min,0.223333,0.180000
25%,0.287667,0.257500
50%,0.310000,0.297500
75%,0.334750,0.322250
max,0.365000,0.380000


### Statistical tests

In [ ]:
nemenyi_p = sp.posthoc_nemenyi_friedman(np.array([df_boosting_dt[col] for col in df_boosting_dt.columns]).T)
nemenyi_p.index = [i for i in df_boosting_dt.columns]
nemenyi_p.columns = [i for i in df_boosting_dt.columns]
nemenyi_p

,dt/10,dt/20
dt/10,1.000000,0.809382
dt/20,0.809382,1.000000


There is a chance of 80.94% of the null hypothesis (the populations are the same) being true. If our threshold to reject the null hipothesis is < 5%, we can conclude that there is enough evidence to reject the null hypothesis. Thus, we accept that the population of the four tested samples are the same. So the best choice is dictated by the greatest mean value, that, in this case is for n_estimators = 10, because the increase in n_estimator to 20 did not change the population and use n_estimators = 20 will result in more computational cost.

## Naive Bayes

In [ ]:
df_boosting_nb = pd.read_csv(path_to_accuracies+'boosting_nb_accuracies.csv')
df_boosting_nb.set_index(['dataset','train/test_type'],inplace=True)
df_boosting_nb.columns = [f'nb/{col}'for col in df_boosting_nb.columns]
df_boosting_nb

nb/10     nb/20
dataset       train/test_type                    
normal_d=1764 Holdout: 70/30   0.210000  0.206667
              Holdout: 80/20   0.345000  0.300000
              Holdout: 90/10   0.250000  0.230000
              10-fold          0.258000  0.281000
normal_d=900  Holdout: 70/30   0.233333  0.263333
              Holdout: 80/20   0.320000  0.350000
              Holdout: 90/10   0.260000  0.090000
              10-fold          0.240000  0.241000
normal_d=459  Holdout: 70/30   0.230000  0.293333
              Holdout: 80/20   0.230000  0.245000
              Holdout: 90/10   0.260000  0.220000
              10-fold          0.216000  0.214000
normal_d=540  Holdout: 70/30   0.316667  0.273333
              Holdout: 80/20   0.245000  0.315000
              Holdout: 90/10   0.250000  0.190000
              10-fold          0.260000  0.297000

### Some stats

In [ ]:
df_boosting_nb.describe()

,nb/10,nb/20
count,16.000000,16.000000
mean,0.257750,0.250604
std,0.038062,0.061248
min,0.210000,0.090000
25%,0.232500,0.218500
50%,0.250000,0.254167
75%,0.260000,0.294250
max,0.345000,0.350000


### Statistical tests

In [ ]:
nemenyi_p = sp.posthoc_nemenyi_friedman(np.array([df_boosting_nb[col] for col in df_boosting_nb.columns]).T)
nemenyi_p.index = [i for i in df_boosting_nb.columns]
nemenyi_p.columns = [i for i in df_boosting_nb.columns]
nemenyi_p

,nb/10,nb/20
nb/10,1.0,0.9
nb/20,0.9,1.0


There is a chance of 90.00% of the null hypothesis (the populations are the same) being true. If our threshold to reject the null hipothesis is < 5%, we can conclude that there is enough evidence to reject the null hypothesis. Thus, we accept that the population of the four tested samples are the same. So the best choice is dictated by the greatest mean value, that, in this case is for n_estimators = 10, because the increase in n_estimator to 20 did not change the population and use n_estimators = 20 will result in more computational cost.

## Comparativo geral

In [ ]:
df_boosting_all = pd.concat([df_boosting_dt,df_boosting_nb],axis=1)
df_boosting_all

dt/10     dt/20     nb/10     nb/20
dataset       train/test_type                                        
normal_d=1764 Holdout: 70/30   0.343333  0.320000  0.210000  0.206667
              Holdout: 80/20   0.365000  0.265000  0.345000  0.300000
              Holdout: 90/10   0.340000  0.230000  0.250000  0.230000
              10-fold          0.299000  0.305000  0.258000  0.281000
normal_d=900  Holdout: 70/30   0.353333  0.310000  0.233333  0.263333
              Holdout: 80/20   0.310000  0.295000  0.320000  0.350000
              Holdout: 90/10   0.310000  0.360000  0.260000  0.090000
              10-fold          0.333000  0.340000  0.240000  0.241000
normal_d=459  Holdout: 70/30   0.223333  0.226667  0.230000  0.293333
              Holdout: 80/20   0.235000  0.235000  0.230000  0.245000
              Holdout: 90/10   0.230000  0.180000  0.260000  0.220000
              10-fold          0.298000  0.275000  0.216000  0.214000
normal_d=540  Holdout: 70/30   0.256667  0.300000  0.316667  0.273333
              Holdout: 80/20   0.320000  0.295000  0.245000  0.315000
              Holdout: 90/10   0.300000  0.380000  0.250000  0.190000
              10-fold          0.321000  0.329000  0.260000  0.297000

In [ ]:
df_boosting_all.describe()

,dt/10,dt/20,nb/10,nb/20
count,16.000000,16.000000,16.000000,16.000000
mean,0.302354,0.290354,0.257750,0.250604
std,0.044328,0.053091,0.038062,0.061248
min,0.223333,0.180000,0.210000,0.090000
25%,0.287667,0.257500,0.232500,0.218500
50%,0.310000,0.297500,0.250000,0.254167
75%,0.334750,0.322250,0.260000,0.294250
max,0.365000,0.380000,0.345000,0.350000


In [ ]:
stats.friedmanchisquare(*[df_boosting_all[col] for col in df_boosting_all.columns])

FriedmanchisquareResult(statistic=5.60126582278481, pvalue=0.1327057073647887)

In [ ]:
algs = ['dt','nb']
df_boosting_10 = df_boosting_all[[f'{alg}/10'for alg in algs]]
nemenyi_p = sp.posthoc_nemenyi_friedman(np.array([df_boosting_all[col] 
                                                  for col in df_boosting_10]).T)
nemenyi_p.index = [i.split('/')[0] for i in df_boosting_10.columns]
nemenyi_p.columns = [i.split('/')[0] for i in df_boosting_10.columns]
nemenyi_p

,dt,nb
dt,1.000000,0.045499
nb,0.045499,1.000000


In [ ]:
algs = ['dt','nb']
df_boosting_10 = df_boosting_all[[f'{alg}/20'for alg in algs]]
nemenyi_p = sp.posthoc_nemenyi_friedman(np.array([df_boosting_all[col] 
                                                  for col in df_boosting_10]).T)
nemenyi_p.index = [i.split('/')[0] for i in df_boosting_10.columns]
nemenyi_p.columns = [i.split('/')[0] for i in df_boosting_10.columns]
nemenyi_p

,dt,nb
dt,1.000000,0.453254
nb,0.453254,1.000000


# Random forests

In [ ]:
df_random_forest = pd.read_csv(path_to_accuracies+'random_forest_accuracies.csv')
df_random_forest.set_index(['dataset','train/test_type'],inplace=True)
df_random_forest.index = df_boosting_dt.index
df_random_forest

random_forest:n_estimators=10,criterion=entropy  \
dataset       train/test_type                                                    
normal_d=1764 Holdout: 70/30                                          0.400000   
              Holdout: 80/20                                          0.335000   
              Holdout: 90/10                                          0.400000   
              10-fold                                                 0.390000   
normal_d=900  Holdout: 70/30                                          0.416667   
              Holdout: 80/20                                          0.405000   
              Holdout: 90/10                                          0.380000   
              10-fold                                                 0.393000   
normal_d=459  Holdout: 70/30                                          0.280000   
              Holdout: 80/20                                          0.230000   
              Holdout: 90/10                                          0.310000   
              10-fold                                                 0.288000   
normal_d=540  Holdout: 70/30                                          0.320000   
              Holdout: 80/20                                          0.350000   
              Holdout: 90/10                                          0.340000   
              10-fold                                                 0.345000   

                               random_forest:n_estimators=100,criterion=gini  \
dataset       train/test_type                                                  
normal_d=1764 Holdout: 70/30                                        0.473333   
              Holdout: 80/20                                        0.495000   
              Holdout: 90/10                                        0.490000   
              10-fold                                               0.556000   
normal_d=900  Holdout: 70/30                                        0.556667   
              Holdout: 80/20                                        0.520000   
              Holdout: 90/10                                        0.560000   
              10-fold                                               0.569000   
normal_d=459  Holdout: 70/30                                        0.320000   
              Holdout: 80/20                                        0.410000   
              Holdout: 90/10                                        0.330000   
              10-fold                                               0.389000   
normal_d=540  Holdout: 70/30                                        0.456667   
              Holdout: 80/20                                        0.490000   
              Holdout: 90/10                                        0.470000   
              10-fold                                               0.514000   

                               random_forest:n_estimators=100,criterion=entropy  \
dataset       train/test_type                                                     
normal_d=1764 Holdout: 70/30                                           0.483333   
              Holdout: 80/20                                           0.495000   
              Holdout: 90/10                                           0.530000   
              10-fold                                                  0.542000   
normal_d=900  Holdout: 70/30                                           0.543333   
              Holdout: 80/20                                           0.525000   
              Holdout: 90/10                                           0.540000   
              10-fold                                                  0.535000   
normal_d=459  Holdout: 70/30                                           0.346667   
              Holdout: 80/20                                           0.365000   
              Holdout: 90/10                                           0.310000   
              10-fold             

### Some stats

In [ ]:
df_random_forest.describe()

,"random_forest:n_estimators=10,criterion=entropy","random_forest:n_estimators=100,criterion=gini","random_forest:n_estimators=100,criterion=entropy","random_forest:n_estimators=10,criterion=gini"
count,16.000000,16.000000,16.000000,16.000000
mean,0.348917,0.474979,0.473229,0.345042
std,0.053413,0.077770,0.077274,0.055820
min,0.230000,0.320000,0.310000,0.243333
25%,0.317500,0.445000,0.450250,0.299000
50%,0.347500,0.490000,0.498500,0.355500
75%,0.394750,0.529000,0.531250,0.385500
max,0.416667,0.569000,0.543333,0.426667


### Statistical tests

In [ ]:
stats.friedmanchisquare(*[df_random_forest[col] for col in df_random_forest.columns])

FriedmanchisquareResult(statistic=37.76582278481011, pvalue=3.1682508155890354e-08)

There is a chance very low chance of the null hypothesis (the populations are the same) being true. If our threshold to reject the null hipothesis is < 5%, we can conclude that there is enough evidence to reject the null hypothesis. Thus, we accept that the population of the four tested samples are the different. And a post-hoc will be conducted

In [ ]:
nemenyi_p = sp.posthoc_nemenyi_friedman(np.array([df_random_forest[col] for col in df_random_forest.columns]).T)
nemenyi_p.index = [i for i in df_random_forest.columns]
nemenyi_p.columns = [i for i in df_random_forest.columns]
nemenyi_p

,"random_forest:n_estimators=10,criterion=entropy","random_forest:n_estimators=100,criterion=gini","random_forest:n_estimators=100,criterion=entropy","random_forest:n_estimators=10,criterion=gini"
"random_forest:n_estimators=10,criterion=entropy",1.000,0.001,0.001,0.900
"random_forest:n_estimators=100,criterion=gini",0.001,1.000,0.900,0.001
"random_forest:n_estimators=100,criterion=entropy",0.001,0.900,1.000,0.001
"random_forest:n_estimators=10,criterion=gini",0.900,0.001,0.001,1.000


The main differences has occur between the number of estimators. Use n_estimators=100 significantly increase the accuracy, so use this value is mandatory. However, the p-value between different criterion (entropy or gini) it is not significant (p-value = 90% >> 5%). So it is indeferent to adopte one or other.

# Stacking

In [ ]:
df_stacking = pd.read_csv(path_to_accuracies+'stacking_accuracies.csv')
df_stacking.set_index(['dataset','train/test_type'],inplace=True)
df_stacking

10     20
dataset       train/test_type              
normal_d=1764 Holdout: 70/30   0.487  0.473
              Holdout: 80/20   0.530  0.560
              Holdout: 90/10   0.510  0.500
              10-fold          0.564  0.573
normal_d=900  Holdout: 70/30   0.600  0.620
              Holdout: 80/20   0.565  0.630
              Holdout: 90/10   0.620  0.560
              10-fold          0.565  0.578
normal_d=459  Holdout: 70/30   0.317  0.343
              Holdout: 80/20   0.360  0.365
              Holdout: 90/10   0.330  0.390
              10-fold          0.314  0.392
normal_d=540  Holdout: 70/30   0.507  0.493
              Holdout: 80/20   0.555  0.525
              Holdout: 90/10   0.450  0.490
              10-fold          0.536  0.536

## Some stats

In [ ]:
df_stacking.describe()

,10,20
count,16.000000,16.000000
mean,0.488125,0.501750
std,0.103058,0.089182
min,0.314000,0.343000
25%,0.427500,0.452750
50%,0.520000,0.512500
75%,0.564250,0.563250
max,0.620000,0.630000


## Statistical tests


In [ ]:
nemenyi_p = sp.posthoc_nemenyi_friedman(np.array([df_stacking[col] for col in df_stacking.columns]).T)
nemenyi_p.index = [i for i in df_stacking.columns]
nemenyi_p.columns = [i for i in df_stacking.columns]
nemenyi_p

,10,20
10,1.000000,0.211299
20,0.211299,1.000000


# Ensemble comparisons

In [ ]:
df_ensemble = pd.concat([df_boosting_all['dt/10'],df_bagging_all['mlp/10'],
                          df_random_forest['random_forest:n_estimators=100,criterion=gini'],df_stacking['10']],axis=1)
df_ensemble

dt/10    mlp/10  \
dataset       train/test_type                       
normal_d=1764 Holdout: 70/30   0.343333  0.500000   
              Holdout: 80/20   0.365000  0.525000   
              Holdout: 90/10   0.340000  0.480000   
              10-fold          0.299000  0.585000   
normal_d=900  Holdout: 70/30   0.353333  0.616667   
              Holdout: 80/20   0.310000  0.625000   
              Holdout: 90/10   0.310000  0.620000   
              10-fold          0.333000  0.575000   
normal_d=459  Holdout: 70/30   0.223333  0.476667   
              Holdout: 80/20   0.235000  0.515000   
              Holdout: 90/10   0.230000  0.500000   
              10-fold          0.298000  0.545000   
normal_d=540  Holdout: 70/30   0.256667  0.486667   
              Holdout: 80/20   0.320000  0.550000   
              Holdout: 90/10   0.300000  0.520000   
              10-fold          0.321000  0.540000   

                               random_forest:n_estimators=100,criterion=gini  \
dataset       train/test_type                                                  
normal_d=1764 Holdout: 70/30                                        0.473333   
              Holdout: 80/20                                        0.495000   
              Holdout: 90/10                                        0.490000   
              10-fold                                               0.556000   
normal_d=900  Holdout: 70/30                                        0.556667   
              Holdout: 80/20                                        0.520000   
              Holdout: 90/10                                        0.560000   
              10-fold                                               0.569000   
normal_d=459  Holdout: 70/30                                        0.320000   
              Holdout: 80/20                                        0.410000   
              Holdout: 90/10                                        0.330000   
              10-fold                                               0.389000   
normal_d=540  Holdout: 70/30                                        0.456667   
              Holdout: 80/20                                        0.490000   
              Holdout: 90/10                                        0.470000   
              10-fold                                               0.514000   

                                  10  
dataset       train/test_type         
normal_d=1764 Holdout: 70/30   0.487  
              Holdout: 80/20   0.530  
              Holdout: 90/10   0.510  
              10-fold          0.564  
normal_d=900  Holdout: 70/30   0.600  
              Holdout: 80/20   0.565  
              Holdout: 90/10   0.620  
              10-fold          0.565  
normal_d=459  Holdout: 70/30   0.317  
              Holdout: 80/20   0.360  
              Holdout: 90/10   0.330  
              10-fold          0.314  
normal_d=540  Holdout: 70/30   0.507  
              Holdout: 80/20   0.555  
              Holdout: 90/10   0.450  
              10-fold          0.536

In [ ]:
df_ensemble.describe()

,dt/10,mlp/10,"random_forest:n_estimators=100,criterion=gini",10
count,16.000000,16.000000,16.000000,16.000000
mean,0.302354,0.541250,0.474979,0.488125
std,0.044328,0.050172,0.077770,0.103058
min,0.223333,0.476667,0.320000,0.314000
25%,0.287667,0.500000,0.445000,0.427500
50%,0.310000,0.532500,0.490000,0.520000
75%,0.334750,0.577500,0.529000,0.564250
max,0.365000,0.625000,0.569000,0.620000


In [ ]:
stats.friedmanchisquare(*[df_ensemble[col] for col in df_ensemble.columns])

FriedmanchisquareResult(statistic=36.81645569620252, pvalue=5.031744582829278e-08)

In [ ]:
nemenyi_p = sp.posthoc_nemenyi_friedman(np.array([df_ensemble[col] for col in df_ensemble.columns]).T)
nemenyi_p.index = [i for i in df_ensemble.columns]
nemenyi_p.columns = [i for i in df_ensemble.columns]
nemenyi_p

,dt/10,mlp/10,"random_forest:n_estimators=100,criterion=gini",10
dt/10,1.000000,0.001000,0.011106,0.001000
mlp/10,0.001000,1.000000,0.031375,0.394338
"random_forest:n_estimators=100,criterion=gini",0.011106,0.031375,1.000000,0.631841
10,0.001000,0.394338,0.631841,1.000000


# Ensemble x simple algorithm

In [ ]:
df_comparison = pd.concat([df_bagging_all['mlp/10'],df_mlp['max_it:1000']],axis=1)
df_comparison

mlp/10  max_it:1000
dataset       train/test_type                       
normal_d=1764 Holdout: 70/30   0.500000     0.486667
              Holdout: 80/20   0.525000     0.530000
              Holdout: 90/10   0.480000     0.540000
              10-fold          0.585000     0.573000
normal_d=900  Holdout: 70/30   0.616667     0.600000
              Holdout: 80/20   0.625000     0.635000
              Holdout: 90/10   0.620000     0.590000
              10-fold          0.575000     0.571000
normal_d=459  Holdout: 70/30   0.476667     0.410000
              Holdout: 80/20   0.515000     0.480000
              Holdout: 90/10   0.500000     0.450000
              10-fold          0.545000     0.485000
normal_d=540  Holdout: 70/30   0.486667     0.503333
              Holdout: 80/20   0.550000     0.515000
              Holdout: 90/10   0.520000     0.450000
              10-fold          0.540000     0.537000

In [ ]:
df_comparison.describe()

,mlp/10,max_it:1000
count,16.000000,16.000000
mean,0.541250,0.522250
std,0.050172,0.061526
min,0.476667,0.410000
25%,0.500000,0.483750
50%,0.532500,0.522500
75%,0.577500,0.571500
max,0.625000,0.635000


In [ ]:
nemenyi_p = sp.posthoc_nemenyi_friedman(np.array([df_comparison[col] for col in df_comparison.columns]).T)
nemenyi_p.index = [i for i in df_comparison.columns]
nemenyi_p.columns = [i for i in df_comparison.columns]
nemenyi_p

,mlp/10,max_it:1000
mlp/10,1.000000,0.045499
max_it:1000,0.045499,1.000000
